- df.drop_duplicates()不指定第一个参数时默认使用全部列作为是否重复的判断依据
- 有的比赛会有两条竞彩记录，让分胜负可能不同，具体操作时取第一条记录，丢弃第二条
- 当筛选条件为具体的值时，可以用==判断，当条件是列表是，可以用.isin()方法
- 按某一列groupby之后，使用get_group()得到的x_df的columns和原始的df.columns一样
- numpy数组的shape，(2,)说明是一维数组,(1,2)说明是二维数组
- 下一步要研究怎么将两个表整合到一起，按主队得分排序后不完全一致

In [25]:
import pandas as pd
import numpy as np
import os
from collections import Counter

In [178]:
files = ['../数据收集/'+f for f in os.listdir('../数据收集/') if '.' not in f]
df_lottery = pd.read_excel(files[5]+'/lottery_results.xlsx')
ids = Counter(df_lottery['betId'])
df_lottery[df_lottery['betId'].isin([key for key ,num in ids.most_common(14)])][:2]

,awayOdds,awayQuarter1Score,awayQuarter2Score,awayScore,awayTeamId,awayTeamName,betId,expertsNum,firstAwayOdds,firstHostOdds,...,jcTotalScore,leagueId,leagueName,leagueType,letScoreTape,matchResult,matchStatus,matchTime,roundType,totalScoreTape
478,20660,34,17,99,551,掘金,77563,0,20660,17510,...,0.0,1,美职男篮,1,15,3,-50,2011-01-31 07:00,常规赛,2075
479,20660,34,17,99,551,掘金,77563,0,20660,17510,...,0.0,1,美职男篮,1,15,3,-50,2011-01-31 07:00,常规赛,2075


In [86]:
df_lottery.drop_duplicates('betId',inplace=True)
df_lottery.shape

(1230, 27)

In [201]:
def merge_one_game(df):
    df_result = pd.DataFrame()
    df_group = df.groupby('GAME_ID')
    for game_id in df['GAME_ID'].unique():
        df_x_game = df_group.get_group(game_id)
        # 主队vs.客队
        host_value = df_x_game[df_x_game['MATCHUP'].str.contains('vs.')].iloc[0].get_values()
        #如果不用iloc[0]则host_value.shape为（1,29），使用iloc[0]之后为(29,)
        away_value = df_x_game[df_x_game['MATCHUP'].str.contains('@')].iloc[0].get_values()
        columns = df_x_game.columns.append(df_x_game.columns.map(lambda x:'Away_'+x))
        df_new = pd.DataFrame(np.concatenate((host_value,away_value)),index=columns)
        df_result = df_result.append(df_new.T)
    return df_result

In [252]:
for f in files[:1]:
    df = pd.read_excel(f+'/games.xlsx')
    df_lottery = pd.read_excel(f+'/lottery_results.xlsx')
    df_lottery.drop_duplicates('betId',inplace=True)
    df_lottery.sort_values(['hostScore','awayScore',''],inplace=True)
    df = merge_one_game(df)
    df.sort_values(['PTS','Away_PTS',''],inplace=True)

In [255]:
a=df_lottery['hostTeamName'].values
f={}
with open('../数据收集/english_name-to-chinese_name.txt','r',encoding='utf-8') as fn:
    for line in fn.read().split('\n'):
        if line:
            e,c = line.split("-")
            f[e]=c
b=[f[name] for name in df['TEAM_NAME'].values]
len([k for i,k in enumerate(a.tolist()) if k!=b[i]])

156